# Data preprocessing

In [59]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [27]:
train_df = pd.read_csv("reddit-comment-classification-comp-551/reddit_train.csv")

In [28]:
test_df = pd.read_csv("reddit-comment-classification-comp-551/reddit_test.csv")

In [42]:
# label each categories
train_df.subreddits = pd.Categorical(train_df.subreddits)
train_df['label'] = train_df.subreddits.cat.codes
label_mapping = dict( enumerate(train_df['subreddits'].cat.categories ) )
train_df

,id,comments,subreddits,label
0,0,"Honestly, Buffalo is the correct answer. I rem...",hockey,11
1,1,Ah yes way could have been :( remember when he...,nba,14
2,2,https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...,leagueoflegends,12
3,3,He wouldn't have been a bad signing if we woul...,soccer,16
4,4,Easy. You use the piss and dry technique. Let ...,funny,9
...,...,...,...,...
69995,69995,"Thank you, you confirm Spain does have nice pe...",europe,8
69996,69996,Imagine how many he would have killed with a r...,leagueoflegends,12
69997,69997,Yes. Only. As in the guy I was replying to was...,canada,6
69998,69998,Looking for something light-hearted or has a v...,anime,4


In [47]:
X_train, X_test, y_train, y_test = train_test_split(train_df['comments'], train_df['label'], random_state=1)

58105    &gt;Is a white writer really not allowed to us...
22589    Yeah! Im English and you can be damned sure I ...
4183     I saw a (I am guessing) 10 year old get hit in...
18670    He was mostly useless every team fight and the...
41299    Good luck actually using that healthcare thoug...
                               ...                        
49100    Add fruit juice in all of its forms to that. S...
20609    Not sure if he's still a "prospect" but Duclai...
21440    ERs in the US can have long wait times too.  I...
50057    My 10 is like someone's 5 and smoke fairly often 
5192     Dee Gordon gets his first homer of the season ...
Name: comments, Length: 52500, dtype: object

6670     Yeah but euron's about to bring cersei tyrion ...
49567    All of his videos are sarcastic and funny...hi...
50796    I love those scenes but it wouldn't have made ...
22310    You do get a smidge of hp for every point of c...
54037    New MMORPG lets you play as someone playing a ...
                               ...                        
32138    Don't want to get into a fight here, but all I...
53648    Its the key that opens the gate to the Obsidia...
64554    That Barry Zito would be a key part to the Gia...
33812    Inb4 triggered Lijang Tower suckers come and c...
30231    Is eating in public illegal too?\n\nIt is a my...
Name: comments, Length: 17500, dtype: object

In [86]:
# vectorize the words

cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)
X_train_cv

<52500x61620 sparse matrix of type '<class 'numpy.int64'>'
	with 960433 stored elements in Compressed Sparse Row format>

In [87]:
X_test_cv = cv.transform(X_test)
X_test_cv

<17500x61620 sparse matrix of type '<class 'numpy.int64'>'
	with 308971 stored elements in Compressed Sparse Row format>

# Running the model with BNB

In [78]:
clf = BernoulliNB()

In [79]:
clf.fit(X_train_cv, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [80]:
preds = clf.predict(X_test_cv)

In [85]:
print('Accuracy score: ', accuracy_score(y_test, preds))

Accuracy score:  0.4948571428571429
